# Research Notes
___

## Testing / Sensor locations
___
1. **B-1:** Rhythmic delta activity with admixed sharp discharges. Spectrogram suggests tail end of a seizure. Disagreement between SZ and “Other”.
2. **B-2:** Frontal lateralized sharp transients at ~1Hz with reversed polarity, likely non-cerebral source. Split between LPD and “Other” (artifact).
3. **B-3:** Diffused semi-rhythmic delta background with low amplitude generalized periodic discharges, shifting morphology, proto-GPD type pattern.
4. **B-4:** Semi-rhythmic delta activity with unstable morphology over the right hemisphere, proto-LRDA pattern.

5. **B-5:** Rhythmic delta activity with unstable morphology, poorly sustained, proto-GRDA.
6. **C-1:** 2Hz LPDs showing evolution with increasing amplitude, pattern between LPDs and seizure onset, edge-case.
7. **D-1:** Abundant GPDs on suppressed background, frequency 1-2Hz, average ~1.5Hz, suggesting a seizure, edge-case.
8. **C-2:** Split between LPDs and GPDs, higher amplitude discharges on the right.
9. **D-2:** Tied between LPDs and LRDA, rhythmic in temporal derivations, periodic in parasagittal derivations.
10. **C-3:** Split between GPDs and LRDA, sharp morphology on ascending limb of delta waves, seen on both sides.
11. **D-3:** Split between GPDs and GRDA, asymmetry in slope between ascending and descending limbs of delta wave, edge-case.
12. **C-4:** Split between LRDA and seizure, 2Hz LRDA on the left, spectrogram suggests tail end of a seizure, edge-case.
13. **D-4:** Split between LRDA and GRDA, higher amplitude rhythmic delta on the left.
14. **C-5:** Split between GRDA and seizure, evolving rhythmic delta activity with embedded epileptiform discharges, edge-case.
15. **D-5:** Split between GRDA and LPDs, generalized rhythmic delta activity, higher amplitude on right with epileptiform discharges, edge-case.
16. **Recording Regions Abbreviations:** LL = left lateral; RL = right lateral; LP = left parasagittal; RP = right parasagittal.

## Abbreviations
___
### EEG Pattern Abbreviations and Their Meanings

- **SIRPIDs**: Stimulus-Induced Rhythmic, Periodic, or Ictal Discharges
  - Common in prolonged EEG monitoring, especially in critically ill patients.
  - Associated with conditions like hypoxic injury, traumatic brain injury, hemorrhage, and toxic-metabolic disturbances.
  - Can be linked to spontaneous electrographic seizures.
  - [Source](https://pubmed.ncbi.nlm.nih.gov/29718832/)

- **SI-PDs**: Stimulus-Induced Periodic Discharges
  - Specific EEG patterns induced by stimuli.

- **SI-RDA**: Stimulus-Induced Rhythmic Delta Activity
  - Rhythmic activity in the delta frequency band induced by a stimulus.

- **SI-SW**: Likely Stimulus-Induced Sharp Waves
  - Sharp waves observed in response to a stimulus.

- **SI-bursts**: Likely Stimulus-Induced Bursts
  - Bursts of electrical activity seen in response to a stimulus.

- **SI-IIC**: Likely Stimulus-Induced Irregular Intermittent Complexes
  - Irregular, intermittent complex discharges in response to stimuli.

- **SI-BIRDS**: Likely Stimulus-Induced Brief Ictal Rhythmic Discharges
  - Brief, seizure-like rhythmic discharges triggered by a stimulus.

These EEG patterns are critical in interpreting the neurological state of patients, particularly those who are critically ill or have specific neurological conditions. They provide valuable information for treatment decisions